<a href="https://colab.research.google.com/github/tck369/1/blob/main/deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 19.0 MB/s eta 0:00:00


In [9]:
from google.colab import files
uploaded = files.upload()


Saving sample_submission.csv to sample_submission.csv
Saving train.csv to train.csv


In [6]:
import optuna
print("✅ Optuna 已成功导入，版本:", optuna.__version__)


✅ Optuna 已成功导入，版本: 4.2.1


In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import optuna
import os


In [10]:
train_df = pd.read_csv('train.csv')
submission_df = pd.read_csv('sample_submission.csv')


In [13]:
def preprocess_data(df, window_size=30, forecast_horizon=1, split_val=True, val_ratio=0.2):
    """
    滑动窗口创建 X, y，并划分训练集和验证集（默认80/20）
    """
    data = df.drop(columns=['ID']).values.astype(np.float32)
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    X_list, y_list = [], []
    for row in data:
        for i in range(len(row) - window_size - forecast_horizon + 1):
            X_list.append(row[i:i + window_size])
            y_list.append(row[i + window_size + forecast_horizon - 1])

    X = np.array(X_list)
    y = np.array(y_list)

    if split_val:
        val_size = int(len(X) * val_ratio)
        return (X[val_size:], y[val_size:], X[:val_size], y[:val_size], scaler)
    else:
        return X, y, scaler


In [22]:
class GenericRNN(nn.Module):
    def __init__(self, model_type, input_size, hidden_size, num_layers, dropout):
        super(GenericRNN, self).__init__()
        self.model_type = model_type

        if model_type == 'RNN':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers,
                              batch_first=True, dropout=dropout)
        elif model_type == 'LSTM':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers,
                               batch_first=True, dropout=dropout)
        else:
            raise ValueError("Unsupported model type")

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        if self.model_type == 'RNN':
            out, _ = self.rnn(x)
        else:
            out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])


In [9]:
def objective(trial):
    window_size = trial.suggest_int("window_size", 20, 50)
    hidden_size = trial.suggest_int("hidden_size", 32, 128)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64])
    model_type = trial.suggest_categorical("model_type", ["RNN", "LSTM"])

    # 获取划分后的训练集和验证集
    X_train, y_train, X_val, y_val, _ = preprocess_data(train_df, window_size, split_val=True)

    # 转成 Tensor
    X_train_tensor = torch.tensor(X_train).unsqueeze(-1)
    y_train_tensor = torch.tensor(y_train).unsqueeze(-1)
    X_val_tensor = torch.tensor(X_val).unsqueeze(-1)
    y_val_tensor = torch.tensor(y_val).unsqueeze(-1)

    train_ds = TensorDataset(X_train_tensor, y_train_tensor)
    val_ds = TensorDataset(X_val_tensor, y_val_tensor)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = GenericRNN(model_type, 1, hidden_size, num_layers, dropout).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # 训练
    model.train()
    for epoch in range(10):  # 可改 epoch
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    # 验证
    model.eval()
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            preds = model(xb).cpu().numpy()
            val_preds.extend(preds)
            val_targets.extend(yb.numpy())

    return mean_squared_error(val_targets, val_preds)


In [10]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)
print("Best Trial:")
print(study.best_trial.params)


[I 2025-03-25 01:32:09,299] A new study created in memory with name: no-name-6c9b6c0a-7ac4-45f8-b56a-a428c476ff48
[I 2025-03-25 01:47:55,437] Trial 0 finished with value: 1.0913966584042551 and parameters: {'window_size': 24, 'hidden_size': 42, 'num_layers': 2, 'dropout': 0.42161524244040866, 'lr': 0.0047430020260399944, 'batch_size': 32, 'model_type': 'LSTM'}. Best is trial 0 with value: 1.0913966584042551.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3356856006445195 and num_layers=1
  warnings.warn(
[I 2025-03-25 01:55:12,098] Trial 1 finished with value: 1.0911392836883886 and parameters: {'window_size': 50, 'hidden_size': 84, 'num_layers': 1, 'dropout': 0.3356856006445195, 'lr': 0.0003488004737571631, 'batch_size': 64, 'model_type': 'LSTM'}. Best is trial 1 with value: 1.0911392836883886.
[I 2025-03-25 02:02:2

Best Trial:
{'window_size': 43, 'hidden_size': 56, 'num_layers': 2, 'dropout': 0.3997308511086487, 'lr': 0.0016642379135249066, 'batch_size': 32, 'model_type': 'LSTM'}


In [11]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)  # 可调试次数，例如50
print("Best Trial:")
print(study.best_trial.params)


[I 2025-03-25 03:25:44,378] A new study created in memory with name: no-name-949cb6a6-c5db-4d6f-a459-25bf025d340b
[I 2025-03-25 03:34:01,648] Trial 0 finished with value: 1.0910671728514554 and parameters: {'window_size': 35, 'hidden_size': 55, 'num_layers': 2, 'dropout': 0.4411309537544822, 'lr': 0.0006427181475330776, 'batch_size': 64, 'model_type': 'LSTM'}. Best is trial 0 with value: 1.0910671728514554.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1443559102226365 and num_layers=1
  warnings.warn(
[I 2025-03-25 03:41:11,770] Trial 1 finished with value: 1.090882244988065 and parameters: {'window_size': 26, 'hidden_size': 94, 'num_layers': 1, 'dropout': 0.1443559102226365, 'lr': 0.0002682235799356061, 'batch_size': 64, 'model_type': 'LSTM'}. Best is trial 1 with value: 1.090882244988065.
[I 2025-03-25 03:54:34,0

Best Trial:
{'window_size': 23, 'hidden_size': 60, 'num_layers': 1, 'dropout': 0.4148422337443821, 'lr': 0.0017535115658193678, 'batch_size': 64, 'model_type': 'LSTM'}


In [17]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# ✅ 使用 GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ 当前使用设备:", device)

# ✅ 获取最佳参数
best_params = study.best_trial.params
window_size = best_params['window_size']
num_layers = best_params['num_layers']
dropout = best_params['dropout'] if num_layers > 1 else 0.0

# ✅ 准备全部训练数据（不划分验证集）
X_all, y_all, scaler = preprocess_data(train_df, window_size, split_val=False)
X_tensor = torch.tensor(X_all).unsqueeze(-1).float().to(device)
y_tensor = torch.tensor(y_all).unsqueeze(-1).float().to(device)

# ✅ 构建模型
model = GenericRNN(
    model_type=best_params['model_type'],
    input_size=1,
    hidden_size=best_params['hidden_size'],
    num_layers=num_layers,
    dropout=dropout
).to(device)

# ✅ 设置优化器与损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=best_params['lr'])
criterion = nn.MSELoss()
loader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=best_params['batch_size'], shuffle=True)

# ✅ 训练模型：100轮
model.train()
for epoch in range(100):
    epoch_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/100 - Loss: {epoch_loss / len(loader):.6f}")


✅ 当前使用设备: cuda
Epoch 1/100 - Loss: 1.000054
Epoch 2/100 - Loss: 0.999943
Epoch 3/100 - Loss: 0.999895
Epoch 4/100 - Loss: 0.999899
Epoch 5/100 - Loss: 0.999886
Epoch 6/100 - Loss: 0.999877
Epoch 7/100 - Loss: 0.999815
Epoch 8/100 - Loss: 0.999806
Epoch 9/100 - Loss: 0.999867
Epoch 10/100 - Loss: 0.999837
Epoch 11/100 - Loss: 0.999827
Epoch 12/100 - Loss: 0.999805
Epoch 13/100 - Loss: 0.999780
Epoch 14/100 - Loss: 0.999761
Epoch 15/100 - Loss: 0.999740
Epoch 16/100 - Loss: 0.999709
Epoch 17/100 - Loss: 0.999719
Epoch 18/100 - Loss: 0.999709
Epoch 19/100 - Loss: 0.999773
Epoch 20/100 - Loss: 0.999680
Epoch 21/100 - Loss: 0.999600
Epoch 22/100 - Loss: 0.999650
Epoch 23/100 - Loss: 0.999632
Epoch 24/100 - Loss: 0.999579
Epoch 25/100 - Loss: 0.999536
Epoch 26/100 - Loss: 0.999469
Epoch 27/100 - Loss: 0.999536
Epoch 28/100 - Loss: 0.999446
Epoch 29/100 - Loss: 0.999450
Epoch 30/100 - Loss: 0.999408
Epoch 31/100 - Loss: 0.999315
Epoch 32/100 - Loss: 0.999345
Epoch 33/100 - Loss: 0.999411
Epoc

In [16]:
from sklearn.preprocessing import StandardScaler
import numpy as np

def preprocess_data(df, window_size=30, forecast_horizon=1, split_val=True, val_ratio=0.2):
    """
    滑动窗口创建 X, y，并划分训练集和验证集（默认80/20）
    """
    data = df.drop(columns=['ID']).values.astype(np.float32)
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    X_list, y_list = [], []
    for row in data:
        for i in range(len(row) - window_size - forecast_horizon + 1):
            X_list.append(row[i:i + window_size])
            y_list.append(row[i + window_size + forecast_horizon - 1])

    X = np.array(X_list)
    y = np.array(y_list)

    if split_val:
        val_size = int(len(X) * val_ratio)
        return (X[val_size:], y[val_size:], X[:val_size], y[:val_size], scaler)
    else:
        return X, y, scaler


In [24]:
# 如果之前没保留 scaler，需要重新获取
_, _, scaler = preprocess_data(train_df, window_size, split_val=False)


In [25]:
# 准备输入数据（取每家公司最后一段）
final_data = train_df.drop(columns=['ID']).values.astype(np.float32)
final_data_scaled = scaler.transform(final_data)

# 模型预测
preds = []
model.eval()
with torch.no_grad():
    for row in final_data_scaled:
        seq = torch.tensor(row[-window_size:]).unsqueeze(0).unsqueeze(-1).float().to(device)
        pred = model(seq).item()
        preds.append(pred)


In [26]:
# 读取原始提交模板
submission_df = pd.read_csv('sample_submission.csv')

# 填入预测值
submission_df['TARGET'] = preds

# 保存提交文件
submission_df.to_csv('submission.csv', index=False)
print("✅ 预测完成，submission.csv 已保存，格式无误！")


✅ 预测完成，submission.csv 已保存，格式无误！


In [28]:
# 构建符合要求的 ID 格式
company_ids = [f"company_{i}" for i in range(442)]

# 构建提交 DataFrame
submission_df = pd.DataFrame({
    'ID': company_ids,
    'value': preds
})

# 保存为 CSV 文件
submission_df.to_csv('submission.csv', index=False)
print("✅ 已按老师要求保存为 submission.csv（ID: company_x, 列名: ID, value）")


✅ 已按老师要求保存为 submission.csv（ID: company_x, 列名: ID, value）


In [29]:
from google.colab import files
files.download('submission.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>